In [1]:
import numpy as np # array operations
import cv2 #opencv library read img operations
import os # use directory & join path
from pathlib import Path # 去掉副檔名

# data visualisation and manipulation
import matplotlib.pyplot as plt #show img and table

from skimage.metrics import mean_squared_error
from skimage.metrics import peak_signal_noise_ratio
from skimage.metrics import structural_similarity

In [2]:
def get_mean_and_std(typeStr, img_name, x):
    x_mean, x_std = cv2.meanStdDev(x)
    x_mean = np.hstack(np.around(x_mean,2))
    x_std = np.hstack(np.around(x_std,2))
    
    # 印出平均與標準差
    if(typeStr == 's'):
        print("\n來源圖檔名: ", img_name)
    elif(typeStr == 't'):
        print("\n目標圖檔名: ", img_name)
    
    print("RGB_channel 平均: ", x_mean)
    print("RGB_channel 標準差: ", x_std)
    
    return x_mean, x_std

In [31]:
def color_transfer(src_rgb, tar_rgb, src_name, tar_name, coltra_name, coltra_num, weight_r, weight_g, weight_b):
    
    s_mean, s_std = get_mean_and_std('s', src_name, src_rgb)
    t_mean, t_std = get_mean_and_std('t', tar_name, tar_rgb)

    # height, width, channel = src_rgb.shape
    
    coltra_rgb = src_rgb

    for k in range(0,3):
        x = src_rgb[:,:,k]
        if(k==0):
            # red channel
            x = ((x-s_mean[k])*(((weight_r*t_std[k])+(1-weight_r)*s_std[k])/s_std[k]))+ weight_r*t_mean[k]+(1-weight_r)*s_mean[k]
        elif(k==1):
            # green channel
            x = ((x-s_mean[k])*(((weight_g*t_std[k])+(1-weight_g)*s_std[k])/s_std[k]))+ weight_g*t_mean[k]+(1-weight_g)*s_mean[k]
        elif(k==2):
            # blue channel
            x = ((x-s_mean[k])*(((weight_b*t_std[k])+(1-weight_b)*s_std[k])/s_std[k]))+ weight_b*t_mean[k]+(1-weight_b)*s_mean[k]

        # # round or +0.5
        # x = round(x)

        # boundary check 超過邊界拉回在邊界上
        x = np.clip(x, 0, 255)
        coltra_rgb[:,:,k] = x
    
    # 存顏色轉換結果圖
    str_weight = '_' + str(weight_r) + '_' + str(weight_g) + '_' + str(weight_b)
    coltra_bgr = cv2.cvtColor(coltra_rgb,cv2.COLOR_RGB2BGR)
    cv2.imwrite('result/'+ coltra_name + str_weight + '.png', coltra_bgr)
    
    return coltra_rgb, s_mean, s_std, t_mean, t_std

In [41]:
if __name__ == '__main__':

    weight_r, weight_g, weight_b = input("Enter 3 channel weight_red , weight_green, weight_blue(0.0 ≤ w ≤ 1.0):").split()
    
    weight_r = float(weight_r)
    weight_g = float(weight_g)
    weight_b = float(weight_b)
    # print("weight is: " + weight_r, weight_g, weight_b)
    # weight_r, weight_g, weight_b = 0.49,0.54,0.62
    global src_rgb

    # 印出所有圖片
    DATASRC = 'source/'
    DATATRG = 'target/'

    # 抓目錄下所有圖檔檔名
    src_name = os.listdir(DATASRC)
    tar_name = os.listdir(DATATRG)

    for i in range (len(src_name)):
            
        # 圖片完整路徑
        src_path = DATASRC + src_name[i]
        tar_path = DATATRG + tar_name[i]
        
        # convert img to array 以彩色格式讀取(三維)
        s_bgr = cv2.imread(src_path ,cv2.IMREAD_COLOR)
        src_bgr = cv2.imread(src_path ,cv2.IMREAD_COLOR)   
        tar_bgr = cv2.imread(tar_path ,cv2.IMREAD_COLOR)
        
        # 原為 BGR 轉為 RGB
        s_rgb = cv2.cvtColor(s_bgr,cv2.COLOR_BGR2RGB)
        src_rgb = cv2.cvtColor(src_bgr,cv2.COLOR_BGR2RGB)
        tar_rgb = cv2.cvtColor(tar_bgr,cv2.COLOR_BGR2RGB)
        
        # 去掉副檔名
        src_name_temp= Path(src_path).stem 
        tar_name_temp= Path(tar_path).stem 
        tar_name_tempp = tar_name_temp.lstrip('0'+str(i+1))
        coltra_name = src_name_temp + tar_name_tempp
        
        # 做色彩轉換
        coltra_rgb, s_mean, s_std, t_mean, t_std = color_transfer(src_rgb, tar_rgb, src_name_temp, tar_name_temp, coltra_name, i+1, weight_r, weight_g, weight_b)
        


來源圖檔名:  01_kodim17
RGB_channel 平均:  [82.68 77.05 69.02]
RGB_channel 標準差:  [46.94 50.11 47.93]

目標圖檔名:  01_kodim23
RGB_channel 平均:  [121.66 109.6   75.79]
RGB_channel 標準差:  [57.45 50.8  53.61]

來源圖檔名:  02_kodim24
RGB_channel 平均:  [113.47 109.55  94.27]
RGB_channel 標準差:  [53.35 54.55 53.11]

目標圖檔名:  02_kodim05
RGB_channel 平均:  [91.5  81.02 67.52]
RGB_channel 標準差:  [49.07 51.4  44.72]

來源圖檔名:  03_kodim07
RGB_channel 平均:  [117.56 110.72  85.37]
RGB_channel 標準差:  [41.08 37.61 39.25]

目標圖檔名:  03_kodim08
RGB_channel 平均:  [127.96 121.08 116.  ]
RGB_channel 標準差:  [63.74 65.29 57.39]

來源圖檔名:  04_miscanthus
RGB_channel 平均:  [134.45 126.99 113.94]
RGB_channel 標準差:  [80.69 77.25 73.34]

目標圖檔名:  04_sunset
RGB_channel 平均:  [125.3  106.91  86.04]
RGB_channel 標準差:  [76.35 63.36 50.01]

來源圖檔名:  05_bed
RGB_channel 平均:  [139.29 135.44 110.14]
RGB_channel 標準差:  [29.56 25.7  57.88]

目標圖檔名:  05_blue
RGB_channel 平均:  [108.12 104.75 121.49]
RGB_channel 標準差:  [89.11 75.67 94.  ]

來源圖檔名:  06_horse
RGB_channel 平